![alt text](./Cerny_logo_1.jpg)

# The effect of ambulance acceleration on mechanical ventilation during neonatal transport

#### Author: Dr Gusztav Belteki

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from matplotlib import dates

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'accelerometer_ventilated'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = os.path.join('/Users', 'guszti', 'ventilation_fabian')

DIR_WRITE = os.path.join(CWD, 'Analyses', TOPIC)

DATA_DUMP = os.path.join('/Volumes', DRIVE, 'data_dump', 'fabian')

# Images will be exported in this subfolder
DIR_WRITE_2 = os.path.join(DATA_DUMP, 'accelero_ventilated_graphs')
if not os.path.isdir(DIR_WRITE_2):
    os.makedirs(DIR_WRITE_2)

In [ ]:
DIR_WRITE, DIR_WRITE_2, DATA_DUMP

### 3. Import ventilator and accelerometer data from pickle archives

In [ ]:
%%time
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_combined'), 'rb') as handle:
    data_pars_combined = pickle.load(handle) 

In [ ]:
%%time
with open('%s/%s.pickle' % (DATA_DUMP, 'ventilator_mode'), 'rb') as handle:
    ventilator_mode = pickle.load(handle)

In [ ]:
%%time
  
with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_2'), 'rb') as handle:
    accelero_ventilated = pickle.load(handle)

with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_low_pass'), 'rb') as handle:
    accelero_ventilated_low_pass = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'accelero_ventilated_high_pass'), 'rb') as handle:
    accelero_ventilated_high_pass = pickle.load(handle)

### 4. Generate and export time series plots for acceleration and ventilation data

### A. 3-dimensional accelerations sampled with 100 Hz

In [ ]:
# Images will be exported in this subfolder
if not os.path.isdir(os.path.join(DIR_WRITE_2, 'time_xyz')):
    os.makedirs(os.path.join(DIR_WRITE_2, 'time_xyz'))

In [ ]:
def accel_time_plot(rec, dpi = 200, filetype = 'jpg', show = True, save = False):
    
    color = ['red', 'blue', 'green']
    fig, ax = plt.subplots(3,1, figsize = (12, 9))
    fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
    
    for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
        accelero_ventilated[rec][axis].plot(ax = ax, color = col, 
            sharex=True, use_index=True, fontsize= 12, x_compat = True)
        ax.grid(True)
        ax.set_ylabel('m/sec$^2$', size = 12)
        ax.set_ylim(-7,7)
        ax.set_xlabel('Time', size  = 12)
        ax.tick_params(which = 'both', labelsize=12)
        ax.legend(loc = 1)
    
        majorFmt = dates.DateFormatter('%H:%M')  
        ax.xaxis.set_major_formatter(majorFmt)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if save:
        fig.savefig('%s/%s_%s_%s.%s' % (DIR_WRITE, 'time_XYZ', rec[0][9:-4], 
            rec[1], filetype), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

    if not show:
        plt.close()

In [ ]:
accel_time_plot(('default__393.txt', 'AL000628'), save = True)

In [ ]:
def accel_time_plot_all(recs, dpi = 200, filetype = 'jpg', show = False, save = True):
    
    color = ['red', 'blue', 'green']
    i = 0
    
    for rec in recs:
    
        fig, ax = plt.subplots(3,1, figsize = (12, 9))
        fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
        for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
            accelero_ventilated[rec][[axis]].plot(ax = ax, color = col, sharex=True, 
                                use_index=True, fontsize= 12, x_compat = True)
            ax.grid(True)
            ax.set_ylabel('m/sec$^2$', size = 12)
            ax.set_ylim(-7,7)
            ax.set_xlabel('Time', size  = 12)
            ax.tick_params(which = 'both', labelsize=12)
            ax.legend(loc = 1)
            
            majorFmt = dates.DateFormatter('%H:%M')  
            ax.xaxis.set_major_formatter(majorFmt)
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
            
        i+=1
        if i != 0 and i % 10 == 0:
            print(datetime.now(), 'Created %s graphs' %i)

        if save:
            fig.savefig('%s/%s/%s_%s_%s.%s' % (DIR_WRITE_2, 'time_XYZ', 'time_XYZ', rec[0][9:-4],
                rec[1], filetype), dpi = dpi, format = filetype,  bbox_inches='tight', pad_inches=0.1);

        if not show:
            plt.close()

### B. 3-dimensional accelerations sampled with 100 Hz after passing through low pass filter

In [ ]:
def accel_time_plot_low_pass(rec, dpi = 200, filetype = 'jpg', show = True, save = False):
    
    color = ['red', 'blue', 'green']
    fig, ax = plt.subplots(3,1, figsize = (12, 9))
    fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
    
    for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
        accelero_ventilated_low_pass[rec][axis].plot(ax = ax, color = col, 
            sharex=True, use_index=True, fontsize= 12, x_compat = True)
        ax.grid(True)
        ax.set_ylabel('m/sec$^2$', size = 12)
        ax.set_ylim(-7,7)
        ax.set_xlabel('Time', size  = 12)
        ax.tick_params(which = 'both', labelsize=12)
        ax.legend(loc = 1)
    
        majorFmt = dates.DateFormatter('%H:%M')  
        ax.xaxis.set_major_formatter(majorFmt)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if save:
        fig.savefig('%s/%s_%s_%s.%s' % (DIR_WRITE, 'time_XYZ_low_pass', rec[0][9:-4], 
            rec[1], filetype), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

    if not show:
        plt.close()

In [ ]:
accel_time_plot_low_pass(('default__393.txt', 'AL000628'), save = True)

In [ ]:
def accel_time_plot_low_pass_all(recs, dpi = 200, filetype = 'jpg', show = False, save = True):
    
    color = ['red', 'blue', 'green']
    i = 0
    
    for rec in recs:
    
        fig, ax = plt.subplots(3,1, figsize = (12, 9))
        fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
        for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
            accelero_ventilated_low_pass[rec][[axis]].plot(ax = ax, color = col, sharex=True, 
                use_index=True, fontsize= 12, x_compat = True)
            ax.grid(True)
            ax.set_ylabel('m/sec$^2$', size = 12)
            ax.set_ylim(-7,7)
            ax.set_xlabel('Time', size  = 12)
            ax.tick_params(which = 'both', labelsize=12)
            ax.legend(loc = 1)
            
            majorFmt = dates.DateFormatter('%H:%M')  
            ax.xaxis.set_major_formatter(majorFmt)
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
            
        i+=1
        if i != 0 and i % 10 == 0:
            print(datetime.now(), 'Created %s graphs' %i)

        if save:
            fig.savefig('%s/%s/%s_%s_%s.%s' % (DIR_WRITE_2, 'time_XYZ', 'time_XYZ_low_pass', rec[0][9:-4],
                rec[1], filetype), dpi = dpi, format = filetype,  bbox_inches='tight', pad_inches=0.1);

        if not show:
            plt.close()

### C. 3-dimensional accelerations sampled with 100 Hz after passing through high pass filter

In [ ]:
def accel_time_plot_high_pass(rec, dpi = 200, filetype = 'jpg', show = True, save = False):
    
    color = ['red', 'blue', 'green']
    fig, ax = plt.subplots(3,1, figsize = (12, 9))
    fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
    
    for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
        accelero_ventilated_high_pass[rec][axis].plot(ax = ax, color = col, 
            sharex=True, use_index=True, fontsize= 12, x_compat = True)
        ax.grid(True)
        ax.set_ylabel('m/sec$^2$', size = 12)
        ax.set_ylim(-7,7)
        ax.set_xlabel('Time', size  = 12)
        ax.tick_params(which = 'both', labelsize=12)
        ax.legend(loc = 1)
    
        majorFmt = dates.DateFormatter('%H:%M')  
        ax.xaxis.set_major_formatter(majorFmt)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    
    if save:
        fig.savefig('%s/%s_%s_%s.%s' % (DIR_WRITE, 'time_XYZ_high_pass', rec[0][9:-4], 
            rec[1], filetype), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

    if not show:
        plt.close()

In [ ]:
accel_time_plot_high_pass(('default__393.txt', 'AL000628'), save = True)

In [ ]:
def accel_time_plot_high_pass_all(recs, dpi = 200, filetype = 'jpg', show = False, save = True):
    
    color = ['red', 'blue', 'green']
    i = 0
    
    for rec in recs:
    
        fig, ax = plt.subplots(3,1, figsize = (12, 9))
        fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)
        for axis, ax, col in zip(['X', 'Y', 'Z'], ax, color):
            accelero_ventilated_high_pass[rec][[axis]].plot(ax = ax, color = col, sharex=True, 
                use_index=True, fontsize= 12, x_compat = True)
            ax.grid(True)
            ax.set_ylabel('m/sec$^2$', size = 12)
            ax.set_ylim(-7,7)
            ax.set_xlabel('Time', size  = 12)
            ax.tick_params(which = 'both', labelsize=12)
            ax.legend(loc = 1)
            
            majorFmt = dates.DateFormatter('%H:%M')  
            ax.xaxis.set_major_formatter(majorFmt)
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
            
        i+=1
        if i != 0 and i % 10 == 0:
            print(datetime.now(), 'Created %s graphs' %i)

        if save:
            fig.savefig('%s/%s/%s_%s_%s.%s' % (DIR_WRITE_2, 'time_XYZ', 'time_XYZ_high_pass', rec[0][9:-4],
                rec[1], filetype), dpi = dpi, format = filetype,  bbox_inches='tight', pad_inches=0.1);

        if not show:
            plt.close()

### D. Unaggregated acceleration data (X, Y, Z) and unaggregated ventilator data

In [ ]:
# Images will be exported in this subfolder
if not os.path.isdir(os.path.join(DIR_WRITE_2, 'accel_vent_plots_unaggregated')):
    os.makedirs(os.path.join(DIR_WRITE_2, 'accel_vent_plots_unaggregated'))

In [ ]:
def accel_vent_plot_unaggregated(recs, dpi = 200, filetype = 'jpg', show = True, save = False):

    pars = ['X', 'Y', 'Z', 'VTemand_kg', 'RR', 'MV_kg', 'PIP', 'FiO2_set'] 
    ylabels = ['m/sec$^2$', 'm/sec$^2$', 'm/sec$^2$', 'mL/kg', '1/min', 'L/kg/min', 'cmH$_2$O', '%']
    legends = ['X', 'Y', 'Z', 'VTemand', 'RR', 'MV', 'PIP', 'FiO$_2$']
    ylimits = [(-6, 6), (-6, 6), (-6, 6), (0, 12), (0, 80), (0, 0.5), (0, 40), (0, 100)]
    color = ['black', 'black', 'black', 'blue', 'green', 'purple', 'red', 'blue']
    
    j = 0
    
    for rec in recs:
        
        if 'RR' not in data_pars_combined[rec].columns:
            continue
        
        else:
    
            fig, axes = plt.subplots(len(pars), 1, figsize = (12, len(pars) * 3))
            fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)

            for i, par in enumerate(pars):
                if i <=2:                     
                    accelero_ventilated[rec][par].plot(ax = axes[i], sharex = True, color = color[i], 
                    alpha = 0.7, linewidth = 1, ylim = ylimits[i], x_compat = True)
        
                else:
                    data_pars_combined[rec][par].plot(ax = axes[i], linewidth = 2, 
                    sharex = True, color = color[i], fontsize= 12, ylim = ylimits[i], x_compat = True)
         
                majorFmt = dates.DateFormatter('%H:%M')  
                axes[i].xaxis.set_major_formatter(majorFmt)
                plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
                         horizontalalignment = 'center')

                axes[i].set_xlabel('Time', size = 12, color = 'black')
                axes[i].set_ylabel(ylabels[i], size = 12, color = 'black')
                axes[i].set_title('',  size = 12, color = 'black')
                axes[i].legend([legends[i]], fontsize = 12)
                axes[i].grid(True, linestyle='-', linewidth=0.5, color = 'gray')

            if save:
                fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, rec[1], 
                '_'.join(pars), filetype), dpi = dpi,  format = filetype, bbox_inches= 'tight', pad_inches=0.1)
        
            if not show:
                plt.close()
            
            j+=1
            if j % 10 == 0:
                print(datetime.now(), 'Created %s graphs' %j)

In [ ]:
accel_vent_plot_unaggregated([('default__393.txt', 'AL000628')], save = True)

In [ ]:
def accel_vent_plot_unaggregated_no_RR(recs, dpi = 200, filetype = 'jpg', show = True, save = False):

    pars = ['X', 'Y', 'Z', 'VTemand_kg', 'MV_kg', 'PIP', 'FiO2_set'] 
    ylabels = ['m/sec$^2$', 'm/sec$^2$', 'm/sec$^2$', 'mL/kg', 'L/kg/min', 'cmH$_2$O', '%']
    legends = ['X', 'Y', 'Z', 'VTemand', 'MV', 'PIP', 'FiO$_2$']
    ylimits = [(-6, 6), (-6, 6), (-6, 6), (0, 12), (0, 0.5), (0, 40), (0, 100)]
    color = ['black', 'black', 'black', 'blue', 'purple', 'red', 'blue']
    
    j = 0
    
    for rec in recs:
        
        fig, axes = plt.subplots(len(pars), 1, figsize = (12, len(pars) * 3))
        fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)

        for i, par in enumerate(pars):
            if i <=2:                     
                accelero_ventilated[rec][par].plot(ax = axes[i], sharex = True, color = color[i], 
                alpha = 0.7, linewidth = 1, ylim = ylimits[i], x_compat = True)
        
            else:
                data_pars_combined[rec][par].plot(ax = axes[i], linewidth = 2, 
                sharex = True, color = color[i], fontsize= 12, ylim = ylimits[i], x_compat = True)
         
            majorFmt = dates.DateFormatter('%H:%M')  
            axes[i].xaxis.set_major_formatter(majorFmt)
            plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
                horizontalalignment = 'center')

            axes[i].set_xlabel('Time', size = 12, color = 'black')
            axes[i].set_ylabel(ylabels[i], size = 12, color = 'black')
            axes[i].set_title('',  size = 12, color = 'black')
            axes[i].legend([legends[i]], fontsize = 12)
            axes[i].grid(True, linestyle='-', linewidth=0.5, color = 'gray')

        if save:
            fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, rec[1], 
            '_'.join(pars), filetype), dpi = dpi,  format = filetype, bbox_inches= 'tight', pad_inches=0.1)
        
        if not show:
            plt.close()
            
        j+=1
        if j % 10 == 0:
            print(datetime.now(), 'Created %s graphs' %j)

In [ ]:
accel_vent_plot_unaggregated_no_RR([('default__393.txt', 'AL000628')])

In [ ]:
def accel_vent_plot_unaggregated_no_RR_all(recs, dpi = 200, filetype = 'jpg', show = False, save = True):

    pars = ['X', 'Y', 'Z', 'VTemand_kg', 'MV_kg', 'PIP', 'FiO2_set'] 
    ylabels = ['m/sec$^2$', 'm/sec$^2$', 'm/sec$^2$', 'mL/kg', 'L/kg/min', 'cmH$_2$O', '%']
    legends = ['X', 'Y', 'Z', 'VTemand', 'MV', 'PIP', 'FiO$_2$']
    ylimits = [(-6, 6), (-6, 6), (-6, 6), (0, 12), (0, 0.5), (0, 40), (0, 100)]
    color = ['black', 'black', 'black', 'blue', 'purple', 'red', 'blue']
    
    j = 0
    
    for rec in recs:
        
        fig, axes = plt.subplots(len(pars), 1, figsize = (12, len(pars) * 3))
        fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)

        for i, par in enumerate(pars):
            if i <=2:                     
                accelero_ventilated[rec][par].plot(ax = axes[i], sharex = True, color = color[i], 
                alpha = 0.7, linewidth = 1, ylim = ylimits[i], x_compat = True)
        
            else:
                data_pars_combined[rec][par].plot(ax = axes[i], linewidth = 2, 
                sharex = True, color = color[i], fontsize= 12, ylim = ylimits[i], x_compat = True)
         
            majorFmt = dates.DateFormatter('%H:%M')  
            axes[i].xaxis.set_major_formatter(majorFmt)
            plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
                horizontalalignment = 'center')

            axes[i].set_xlabel('Time', size = 12, color = 'black')
            axes[i].set_ylabel(ylabels[i], size = 12, color = 'black')
            axes[i].set_title('',  size = 12, color = 'black')
            axes[i].legend([legends[i]], fontsize = 12)
            axes[i].grid(True, linestyle='-', linewidth=0.5, color = 'gray')

        if save:
            fig.savefig('%s/%s/%s_%s.%s' % (DIR_WRITE_2, 'accel_vent_plots_unaggregated', rec[1], 
            '_'.join(pars), filetype), dpi = dpi,  format = filetype, bbox_inches= 'tight', pad_inches=0.1)
        
        if not show:
            plt.close()
            
        j+=1
        if j % 10 == 0:
            print(datetime.now(), 'Created %s graphs' %j)

# Figures for the paper

### Supplementary Figure 2

In [ ]:
rec = 'default__393.txt', 'AL000628'
dpi = 300
filetype = 'jpg'
majorFmt = dates.DateFormatter('%H:%M')  

pars = ['X', 'Y', 'Z', 'VTemand_kg', 'RR', 'MV_kg', 'PIP', 'FiO2_set'] 
ylabels = ['m/sec$^2$', 'm/sec$^2$', 'm/sec$^2$', 'mL/kg', '1/min', 'L/kg/min', 'cmH$_2$O', '%']
legends = ['X', 'Y', 'Z', 'VTemand', 'RR', 'MV', 'PIP', 'FiO$_2$']
ylimits = [(-6, 6), (-6, 6), (-6, 6), (0, 12), (0, 80), (0, 0.5), (0, 40), (0, 100)]
color = ['black', 'black', 'black', 'blue', 'green', 'purple', 'red', 'blue']
    
fig, axes = plt.subplots(len(pars), 1, figsize = (9, len(pars) * 3))
fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=0, hspace=None)

for i, par in enumerate(pars):
    if i <=2:                     
        accelero_ventilated[rec][par].plot(ax = axes[i], sharex = True, color = color[i], 
            alpha = 0.7, linewidth = 1, ylim = ylimits[i], x_compat = True)
        
    else:
        data_pars_combined[rec][par].plot(ax = axes[i], linewidth = 2, 
        sharex = True, color = color[i], fontsize= 12, ylim = ylimits[i], x_compat = True)
         
  
    axes[i].xaxis.set_major_formatter(majorFmt)
    plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    axes[i].set_xlabel('Time', size = 12, color = 'black')
    axes[i].set_ylabel(ylabels[i], size = 12, color = 'black')
    axes[i].set_title('',  size = 12, color = 'black')
    axes[i].legend([legends[i]], fontsize = 12)
    axes[i].grid(True, linestyle='-', linewidth=0.5, color = 'gray')

    fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_2', filetype), 
        dpi = dpi,  format = filetype, bbox_inches= 'tight', pad_inches=0.1)
        

### Supplementary Figure 4

In [ ]:
rec = 'default__393.txt', 'AL000628'
dpi = 300
filetype = 'pdf'
majorFmt = dates.DateFormatter('%H:%M')  

pars = ['X', 'Y', 'Z',] 
ylabels = ['m/sec$^2$', 'm/sec$^2$', 'm/sec$^2$']
legends = ['X', 'Y', 'Z',]
    
fig, axes = plt.subplots(len(pars), 2, figsize = (18, len(pars) * 3))
fig.subplots_adjust(left=None, bottom=0, right=None, top=None, wspace=None, hspace=None)

for i, par in enumerate(pars):
    accelero_ventilated_low_pass[rec][par].plot(ax = axes[i,0], sharex = True, color = 'black', 
        alpha = 0.7, linewidth = 1, ylim = [-6, 6], x_compat = True)
   
    axes[i,0].xaxis.set_major_formatter(majorFmt)
    plt.setp(axes[i,0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    axes[i,0].set_xlabel('Time', size = 12, color = 'black')
    axes[i,0].set_ylabel(ylabels[i], size = 12, color = 'black')
    axes[i,0].set_title('',  size = 12, color = 'black')
    axes[i,0].legend([legends[i]], fontsize = 12)
    axes[i,0].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
 
for i, par in enumerate(pars):
    accelero_ventilated_high_pass[rec][par].plot(ax = axes[i,1], sharex = True, color = 'black', 
        alpha = 0.7, linewidth = 1, ylim = [-6, 6], x_compat = True)
   
    axes[i,1].xaxis.set_major_formatter(majorFmt)
    plt.setp(axes[i,1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    axes[i,1].set_xlabel('Time', size = 12, color = 'black')
    axes[i,1].set_ylabel(ylabels[i], size = 12, color = 'black')
    axes[i,1].set_title('',  size = 12, color = 'black')
    axes[i,1].legend([legends[i]], fontsize = 12)
    axes[i,1].grid(True, linestyle='-', linewidth=0.5, color = 'gray')

    fig.text(0.1, 0.97, 'A', fontsize = 16); fig.text(0.52, 0.97, 'B', fontsize = 16)
    
    fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3', filetype), 
        dpi = dpi,  format = filetype, bbox_inches= 'tight', pad_inches=0.1)

